<a href="https://colab.research.google.com/github/BaBa0525/machine-learning/blob/main/HW5/hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import package

In [1]:
import csv
import cv2
import numpy as np
import random
import os

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

## Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab\ Notebooks/ML-HW5
!ls

/content/drive/MyDrive/Colab Notebooks/ML-HW5
captcha-hacker.zip  hw5.ipynb  input


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Check dataset

In [5]:
import os

for dirname, _, filenames in os.walk('./input'):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))
    if len(filenames) > 3:
        print("...")


./input/sample_submission.csv
./input/test/task1/p1mQMrocXHhSDTxe.png
./input/test/task1/wH6XHdKbYBL7l29q.png
./input/test/task1/zIUgMdE4RUJ2XWDI.png
...
./input/test/task2/hIGVIh29SGCmyllJ.png
./input/test/task2/mVC7Bn9xc4I7X9ZC.png
./input/test/task2/kkiDDFzu8iPBOuDW.png
...
./input/test/task3/3hTBBm1cGTSdNNDl.png
./input/test/task3/42AwuKvLKi2F636B.png
./input/test/task3/AYuafmx4qou5HsGd.png
...
./input/train/annotations.csv
./input/train/task1/ixR3Su7WAzEEn584.png
./input/train/task1/Z16DOd7u1x5Rh9uo.png
./input/train/task1/o2lcHmf6OyAtCuY8.png
...
./input/train/task2/GJtENwld5UvVcPcB.png
./input/train/task2/UIwaYwOchgxtV5zN.png
./input/train/task2/BZiz0GWTxkTPsgbV.png
...
./input/train/task3/DME5yC43SSVtdLyk.png
./input/train/task3/NXg03BRj0yxP9Pf2.png
./input/train/task3/PHUbZrDwWJrQ9m3J.png
...


In [6]:
TRAIN_PATH = "./input/train"
TEST_PATH = "./input/test"

## Dataset Class

In [7]:
class Task1Dataset(Dataset):
  def __init__(self, data, root, return_filename=False):
    self.data = [sample for sample in data if sample[0].startswith("task1")]
    self.return_filename = return_filename
    self.root = root

  def __getitem__(self, index):
    filename, label = self.data[index]
    img = cv2.imread(f"{self.root}/{filename}")
    img = cv2.resize(img, (32, 32))
    img = np.mean(img, axis=2)
    if self.return_filename:
      return torch.FloatTensor((img - 128) / 128), filename

    return torch.FloatTensor((img - 128) / 128), int(label)

  def __len__(self):
    return len(self.data)
    

## Model Class

In [8]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(1024, 512),
        nn.LeakyReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    b, h, w = x.shape
    x = x.view(b, h*w)
    return self.layers(x)

## Process training data

In [9]:
train_data = []
val_data = []

with open(f"{TRAIN_PATH}/annotations.csv", newline="") as csvfile:
  for row in csv.reader(csvfile, delimiter=","):
    if random.random() < 0.7:
      train_data.append(row)
    else:
      val_data.append(row)

train_ds = Task1Dataset(train_data, TRAIN_PATH)
train_dl = DataLoader(train_ds, batch_size=500, drop_last=True, shuffle=True)

val_ds = Task1Dataset(val_data, TRAIN_PATH)
val_dl = DataLoader(val_ds, batch_size=500, drop_last=True, shuffle=True)

## Training

In [10]:
model = Model().to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(100):
  print(f"Epoch: [{epoch}]")

  model.train()
  for image, label in train_dl:
    image = image.to(device)
    label = label.to(label)

    pred = model(image)
    loss = loss_fn(pred, label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  sample_count = 0
  correct_count = 0

  model.eval()
  for image, label in val_dl:
    image = image.to(device)
    label = label.to(label)

    pred = model(image)
    loss = loss_fn(pred, label)

    pred = torch.argmax(pred, dim=1)

    sample_count += len(image)
    correct_count += (label == pred).sum()

  print(f"accuracy(validation): {correct_count / sample_count}")



Epoch: [0]
accuracy(validation): 0.09399999678134918
Epoch: [1]
accuracy(validation): 0.10400000214576721
Epoch: [2]
accuracy(validation): 0.0860000029206276
Epoch: [3]
accuracy(validation): 0.09399999678134918
Epoch: [4]
accuracy(validation): 0.164000004529953
Epoch: [5]
accuracy(validation): 0.2680000066757202
Epoch: [6]
accuracy(validation): 0.1860000044107437
Epoch: [7]
accuracy(validation): 0.1899999976158142
Epoch: [8]
accuracy(validation): 0.2980000078678131
Epoch: [9]
accuracy(validation): 0.31200000643730164
Epoch: [10]
accuracy(validation): 0.335999995470047
Epoch: [11]
accuracy(validation): 0.32199999690055847
Epoch: [12]
accuracy(validation): 0.3140000104904175
Epoch: [13]
accuracy(validation): 0.39800000190734863
Epoch: [14]
accuracy(validation): 0.4059999883174896
Epoch: [15]
accuracy(validation): 0.36399999260902405
Epoch: [16]
accuracy(validation): 0.33799999952316284
Epoch: [17]
accuracy(validation): 0.4359999895095825
Epoch: [18]
accuracy(validation): 0.43399998545646

## Output prediction

In [20]:
test_data = []

with open(f"{TEST_PATH}/../sample_submission.csv", newline="") as csvfile:
  for row in csv.reader(csvfile, delimiter=","):
    test_data.append(row)

test_ds = Task1Dataset(test_data, TEST_PATH, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=500, num_workers=2, drop_last=False, shuffle=False)

if os.path.exists("submission.csv"):
  fo = open("submission.csv", mode="a", newline="")
  csv_writer = csv.writer(fo)
else:
  fo = open("submission.csv", mode="a", newline="")
  csv_writer = csv.writer(fo)
  csv_writer.writerow(["filename", "label"])

model.eval()
for image, filenames in test_dl:
  image = image.to(device)

  pred = model(image)
  pred = torch.argmax(pred, dim=1)

  for ind, filename in enumerate(filenames):
    if (len(filename) < 3):
      print(filename)
    csv_writer.writerow([filename, str(pred[ind].item())])



In [21]:
for filename, _ in test_data:
  if filename.startswith("task2") or filename.startswith("task3"):
    # print(filename)
    csv_writer.writerow([filename, 0])

fo.close()

In [22]:
!ls

captcha-hacker.zip  hw5.ipynb  input  submission.csv
